In [42]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import gc
import math

##  function Extract_NER():   Extract NER from receiving text list

In [43]:
def Extract_NER(ls_textRow):
    from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
    
    # 載入 自訂詞典
    User_Dict = {}
    with open("C:/Users/veniv/OneDrive/桌面/NSYSU1092-20210601T123714Z-001/NSYSU1092/Lab/KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
        for udic in UDicts:
            udWord = udic.strip().split(" ")
            if len(udWord) > 1:
                User_Dict[udWord[0]] = udWord[1]
            else:
                User_Dict[udWord[0]] = 10    # 未給定權重者一律賦予預設值 10                
    dictionary = construct_dictionary(User_Dict)
 
    ws = WS("C:/Users/veniv/OneDrive/桌面/NSYSU1092-20210601T123714Z-001/NSYSU1092/Lab/KCC Data/data")
    pos = POS("C:/Users/veniv/OneDrive/桌面/NSYSU1092-20210601T123714Z-001/NSYSU1092/Lab/KCC Data/data")
    ner = NER("C:/Users/veniv/OneDrive/桌面/NSYSU1092-20210601T123714Z-001/NSYSU1092/Lab/KCC Data/data")
    
    #  ls_segRow = ws(ls_textRow) ------------   without User Dictionary
    ls_segRow = ws(ls_textRow, 
                   sentence_segmentation=True,
                   segment_delimiter_set = {",", "。", ":", "?", "!", ";", "、"},
                   coerce_dictionary = dictionary)
    #print("ls_segRow = ", ls_segRow)
    #print("-----------------------------")
    
    del ws
    del WS
    gc.collect()
    ls_posRow = pos(ls_segRow)
    del pos
    del POS
    gc.collect()
    ls_nerRow = ner(ls_segRow,ls_posRow)
    del ner
    del NER
    gc.collect()
    #print("ls_nerRow = ", ls_nerRow)
    #print("-----------------------------")
    return ls_nerRow

## 1. Read data from csv or xlsx file

In [44]:
#讀檔

df_file = pd.read_csv("C:/Users/veniv/OneDrive/桌面/NSYSU1092-20210601T123714Z-001/NSYSU1092/Lab/KCC Data/NewsCisTest.csv")
print(df_file.columns)
print("-----------------------------------------------------------")
df_file.head(3)

Index(['Seq', 'Date', 'Text'], dtype='object')
-----------------------------------------------------------


,Seq,Date,Text
0,1,2016/4/18,馬英九總統昨天上午視察衛武營藝術文化中心，文化部長洪孟啟、工程會副主委顏久榮及高雄市副市長吳...
1,2,2016/4/18,日本九州熊本縣接連遭強震襲擊，台灣分別捐贈熊本縣政府和日本政府款項，高雄市長陳菊、台南市長賴...
2,3,2016/4/18,壽山動物園為增進動物飼養福祉及保育工作，特設立「高雄市壽山動物園認養辦法」設立動物認養專戶，...


## 2.  Main function:
### 2.1  Pass text data to function "Extract_NER" by row
### 2.2  Integrate every 10 NER return rows into one data frame  
### 2.3  Write frame into small csv after droping duplicate, locating class, filtering short NER, sorting by class.. 

In [45]:
#逐筆傳遞欄位 "text"
df_text = df_file["Text"]
print("Text 欄位筆數 = ",len(df_text))
print("------------------------------------------")
ls_class = list()
ls_ner = list()
fileCunt = 0
rowCunt = 0
for i in range(len(df_text)):
    rowCunt += 1
    ls_dfRow = [df_text[i]]
    # print("ls_dfRow = ", ls_dfRow)
    print("Row No = ",str(i))
    entity_list = Extract_NER(ls_dfRow)
    for j in range(len(entity_list[0])):
        seg = entity_list[0].pop()
        # print("seg = ",seg)
        ls_class.append(seg[2])
        ls_ner.append(seg[3])
    #print("===========================================")
    #print("ls_class = ",ls_class)
    #print("ls_ner = ",ls_ner)
    # if i == 5 or rowCunt == 2:
    if i == (len(df_text)-1) or rowCunt == 10:
        fileCunt += 1
        extract_df = pd.DataFrame({'Class':ls_class, 'NER':ls_ner})
        deDup_df = extract_df.drop_duplicates().copy()
        del extract_df
        cate_df = deDup_df.loc[deDup_df['Class'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE','NORP'])].copy()
        del deDup_df
        filt_df = cate_df[cate_df['NER'].map(len) >= 2].copy()
        del cate_df
        NER_df = filt_df.sort_values(by=['Class']).copy()
        del filt_df
        #print("__________________________________________________________")  
        #print(NER_df)
        print("fileCunt No = ",str(fileCunt))
        #NER_df.to_csv(r"News Data\News NER Out\\NewsNER870_" + str(fileCunt)+ ".csv")
        NER_df.to_csv(r"ner_end" + str(fileCunt)+ ".csv")
        rowCunt = 0
print("=====================================================================")
print("Totle NER Files = ",fileCunt)


Text 欄位筆數 =  15
------------------------------------------
Row No =  0
Row No =  1
Row No =  2
Row No =  3
Row No =  4
Row No =  5
Row No =  6
Row No =  7
Row No =  8
Row No =  9
fileCunt No =  1
Row No =  10
Row No =  11
Row No =  12
Row No =  13
Row No =  14
fileCunt No =  2
Totle NER Files =  2


## 3. Integrate all csv files into single data frame

In [76]:
import glob
import pandas as pd

#path = r'News Data\News NER OUT'
#path = r'nerout'
#allFiles = glob.glob("{}/*.csv".format(path))
allFiles = glob.glob("./*.csv")
li = []
for filename in allFiles:
    df = pd.read_csv(filename, index_col=0, header=1)
    li.append(df)
    print(li)

nerFrame = pd.concat(li, axis=0, ignore_index=True)
print(len(nerFrame))
nerFrame.groupby("EVENT").count()

[      EVENT   二次大戰
156               
170     GPE     越南
52      GPE     日本
73      GPE   中華民國
74      GPE  九州熊本縣
82      GPE     台北
..      ...    ...
281  PERSON    沈英章
31   PERSON    簡文彬
276  PERSON    明華園
315  PERSON    陳宜民
42   PERSON    洪孟啟

[120 rows x 2 columns]]
[      EVENT   二次大戰
156               
170     GPE     越南
52      GPE     日本
73      GPE   中華民國
74      GPE  九州熊本縣
82      GPE     台北
..      ...    ...
281  PERSON    沈英章
31   PERSON    簡文彬
276  PERSON    明華園
315  PERSON    陳宜民
42   PERSON    洪孟啟

[120 rows x 2 columns],       EVENT 九州熊本強震
474               
156   EVENT   二次大戰
392   EVENT    水門案
505   EVENT   熊本地震
552   EVENT   普度法會
322     GPE    高雄縣
..      ...    ...
408  PERSON     福特
403  PERSON    布希特
402  PERSON     阿扁
70   PERSON    馬英九
580  PERSON     仙官

[206 rows x 2 columns]]
326


,二次大戰,九州熊本強震
EVENT,,
EVENT,0,4
GPE,24,31
LAW,1,2
LOC,1,1
NORP,6,7
ORG,60,74
PERSON,28,87


## 4. Drop duplicates, group by class and write into csv file 

In [77]:
ddFrame = nerFrame.drop_duplicates().copy()
print(len(ddFrame))
ddFrame.groupby('EVENT').count()

326


,二次大戰,九州熊本強震
EVENT,,
EVENT,0,4
GPE,24,31
LAW,1,2
LOC,1,1
NORP,6,7
ORG,60,74
PERSON,28,87


In [84]:
# ddFrame.sort_values(by=['Class']).to_csv(r"News Data\News NER OUT\\NewsNER.csv")
ddFrame.sort_values(by=['EVENT']).to_csv(r"C:/Users/veniv/Downloads/nerout/ner_end.csv")